<a href="https://colab.research.google.com/github/moneebullah25/TeacherAssistant/blob/main/Teacher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# CONV_CHAT_BOT Setup

In [3]:
%%capture
!pip install langchain
!pip install pypdf
!pip install sentence_transformers
!pip install chromadb
!pip install accelerate
!pip install --upgrade accelerate
!pip install bitsandbytes

In [2]:
import os
from langchain.document_loaders import PyPDFLoader

path = "/content/drive/MyDrive/Books"
documents = []
for file in os.listdir(path):
    if file.endswith('.pdf'):
        pdf_path = path+'/' + file
        print(pdf_path)
        loader = PyPDFLoader(pdf_path)
        documents.extend(loader.load())

/content/drive/MyDrive/Books/Basic_Marketing_2021.pdf
/content/drive/MyDrive/Books/Marketing_Book.pdf


/usr/local/lib/python3.10/dist-packages/pypdf/_cmap.py:192: PdfReadWarning: Advanced encoding [] not implemented yet
  warnings.warn(


In [3]:
from langchain.docstore.document import Document
from langchain.vectorstores import Chroma
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# Load multiple documents
# pdf_files = ["/content/drive/MyDrive/Books/Basic_Marketing_2021.pdf", "/content/drive/MyDrive/Books/Marketing_Book.pdf"]

# Create an empty list to store documents
all_documents = []
all_documents.extend(documents)
# Load documents using PyPDFLoader
# for pdf_file in pdf_files:
#     loader = PyPDFLoader(pdf_file)
#     documents = loader.load()
#     all_documents.extend(documents)

# Split the documents
text_splitter = CharacterTextSplitter(chunk_size=400, chunk_overlap=40)
all_documents = text_splitter.split_documents(all_documents)

# Initialize HuggingFaceEmbeddings
hf_embed = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Create Chroma vector store from multiple documents
Vector_db = Chroma.from_documents(collection_name="document_docs", documents=all_documents, embedding=hf_embed, persist_directory="/content/drive/MyDrive/Conv_bot/Data/Docs/Persist_dir")

# Perform a similarity search (if needed)
# Vector_db.similarity_search("dummy")  # Tickle it to persist metadata (?)

# Persist the vector store
Vector_db.persist()


#1/ Download our 2 embeddings model from hugging face
(same as data preparation)

In [4]:
def get_available_gpus():
    """Get a list of available gpu devices (formatted as strings)."""
    from tensorflow.python.client import device_lib
    local_device_protos = device_lib.list_local_devices()
    print(local_device_protos)
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

In [5]:
# Start here to load a previously-saved DB
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

if len(get_available_gpus()) == 0:
  Exception("Running dolly without GPU will be slow. We recommend you switch to a Single Node cluster with at least 1 GPU to properly run this demo.")

# gardening_vector_db_path = "/dbfs"+demo_path+"/vector_db"
vector_db_path = "/content/drive/MyDrive/Conv_bot/Data/Docs/db_path"

hf_embed = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
db = Chroma(collection_name="document_docs", embedding_function=hf_embed, persist_directory=vector_db_path)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15715159457187795362
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11528896512
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5433651596664286137
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [6]:
def get_similar_docs(question, similar_doc_count):
    similar_docs = Vector_db.similarity_search(question, similar_doc_count)

    unique_docs = []
    seen_doc_contents = set()

    for doc in similar_docs:
        doc_content = doc.page_content  # Assuming 'page_content' is the key for the document content
        if doc_content not in seen_doc_contents:
            unique_docs.append(doc)
            seen_doc_contents.add(doc_content)

    return unique_docs


# Let's test it with blackberries:
for doc in get_similar_docs("What is GPU?", 3):
    print(doc)

page_content="packed. It got very high ratings as 5/5 at Think2Buy and considered a\ndesign marvel . Asus Project Precog (Future Plan) Asus is after dual-screen\nthat won't have a keyboard. If this becomes successful then perhaps in not\nso distant future we all shall be using keyboard-less laptops. It is an Al\npowered laptop that can be used either with its own one screen convertible\nto virtual keyboard or alternately with a bluetooth connected keyboard if\nboth screens are needed. 218" metadata={'page': 282, 'source': '/content/drive/MyDrive/Books/Basic_Marketing_2021.pdf'}


# 3/ Prompt engineering with langchain
Now we can compose with a language model and prompting strategy to make a langchain chain that answers questions.

In [7]:
template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

  Instruction:
  You are a marketing teacher at university. Your job is to provide answers to student questions.
  Use only information in the following paragraphs to answer the question at the end.
  Explain the answer with reference to these paragraphs.
  If you don't have the information in below paragraphs then give response "I will get back to you on this".

  {context}

  Question: {question}

  Response:
  """

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from langchain import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.chains.question_answering import load_qa_chain

def build_qa_chain():
  torch.cuda.empty_cache()
  model_name = "databricks/dolly-v2-3b" # can use dolly-v2-3b or dolly-v2-7b for smaller model and faster inferences.

  # Increase max_new_tokens for a longer response
  # Other settings might give better results! Play around
  instruct_pipeline = pipeline(model=model_name, torch_dtype=torch.float16, trust_remote_code=True, device_map="auto", return_full_text=True, max_new_tokens=512, top_p=0.95, top_k=50,model_kwargs={'load_in_8bit': True})
  # Note: if you use dolly 12B or smaller model but a GPU with less than 24GB RAM, use 8bit. This requires %pip install bitsandbytes
  # instruct_pipeline = pipeline(model=model_name, trust_remote_code=True, device_map="auto", model_kwargs={'load_in_8bit': True})
  # For GPUs without bfloat16 support, like the T4 or V100, use torch_dtype=torch.float16 below
  model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16, trust_remote_code=True)
  prompt = PromptTemplate(input_variables=['context', 'question'], template=template)

  hf_pipe = HuggingFacePipeline(pipeline=instruct_pipeline)
  # Set verbose=True to see the full prompt:
  return load_qa_chain(llm=hf_pipe, chain_type="stuff", prompt=prompt, verbose=False)

In [9]:
qa_chain = build_qa_chain()

#4/ Using the Chain for Simple Question Answering
That's it! It's ready to go. Define a function to answer a question and pretty-print the answer, w

In [10]:
def displayHTML(html):
    """Display HTML in Jupyter notebook."""
    from IPython.display import HTML
    display(HTML(html))


def answer_question(question):
  similar_docs = get_similar_docs(question, similar_doc_count=3)
  result = qa_chain({"input_documents": similar_docs, "question": question})
  if result is None:
    result_html = f'<h1>Couldnt found data from book</h1>'
    return
  # print(result)
  result_html = f"<p><blockquote style=\"font-size:24\">{question}</blockquote></p>"
  result_html += f"<p><blockquote style=\"font-size:18px\">{result['output_text']}</blockquote></p>"
  result_html += "<p><hr/></p>"
  for d in result["input_documents"]:
    source_id = d.metadata["source"]
    page = d.metadata['page']
    result_html += f"<p><blockquote>{d.page_content}<br/>(Source: <p>{source_id} Page: {page}</p>)</blockquote></p>"
  displayHTML(result_html)



In [11]:
answer_question("If I query you about something which you don't know what will be your response?")

In [12]:
answer_question("I have exam tomorrow on Marketing. What should I do to pass the exam?")

In [13]:
answer_question("I know that marketing is to sell things to people through various means. Is it enough to pass the course?")

In [14]:
answer_question("What is Postmodernism?")

In [16]:
answer_question("What is marketing?")

In [18]:
answer_question("Explain in detail what is marketing? Also add some examples")